#### Aquí encontrarás todo lo que necesitas saber: https://www.kaggle.com/t/ab8726f0cfc84544abbae69a6be88071

## Librerias

In [9]:
import pandas as pd
import numpy as np
import utils.DataBase as db
import re
import utils.Functions as fn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from warnings import filterwarnings
filterwarnings("ignore")

import urllib.request
from PIL import Image

## Data

In [10]:
### Para que funcione necesitas bajarte los archivos de datos de Kaggle 
train_tb = pd.read_csv("data/train.csv")


## Data exploration

In [11]:
train_tb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         912 non-null    int64  
 1   Company           912 non-null    object 
 2   Product           912 non-null    object 
 3   TypeName          912 non-null    object 
 4   Inches            912 non-null    float64
 5   ScreenResolution  912 non-null    object 
 6   Cpu               912 non-null    object 
 7   Ram               912 non-null    object 
 8   Memory            912 non-null    object 
 9   Gpu               912 non-null    object 
 10  OpSys             912 non-null    object 
 11  Weight            912 non-null    object 
 12  Price_euros       912 non-null    float64
dtypes: float64(2), int64(1), object(10)
memory usage: 92.8+ KB


In [12]:
train_tb.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1104,Acer,Aspire ES1-523,Notebook,15.6,1366x768,AMD A8-Series 7410 2.2GHz,4GB,500GB HDD,AMD Radeon R5,Windows 10,2.4kg,387.0
1,114,Dell,XPS 13,Ultrabook,13.3,Quad HD+ / Touchscreen 3200x1800,Intel Core i7 7560U 2.4GHz,8GB,256GB SSD,Intel Iris Plus Graphics 640,Windows 10,1.23kg,1379.0
2,172,Acer,Aspire A517-51G,Notebook,17.3,IPS Panel Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,256GB SSD,Nvidia GeForce MX150,Windows 10,3kg,854.0
3,918,HP,Elitebook 820,Netbook,12.5,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,512GB SSD,Intel HD Graphics 620,Windows 10,1.26kg,1483.0
4,447,Lenovo,IdeaPad 320-15AST,Notebook,17.3,1600x900,AMD A6-Series 9220 2.5GHz,8GB,1TB HDD,AMD Radeon R4,Windows 10,2.8kg,519.0


## Data processing

In [13]:
# Establece la columna laptop_ID como índice
train_tb.set_index("laptop_ID", inplace=True)

# Cambia el tipo de datos de la columna Company
train_tb["Company"] = train_tb["Company"].astype("category")

# Cambia el tipo de datos de la columna TypeName
train_tb["TypeName"] = train_tb["TypeName"].astype("category")

# Crea una nueva columna llamada Touchscreen que determina aquellos laptops que tienen Touchscreen
train_tb["Touchscreen"] = train_tb["ScreenResolution"].str.contains(
    ".*Touchscreen.*")
train_tb["Touchscreen"] = train_tb["Touchscreen"].astype("int")

# Cambia el tipo de datos de la columna ScreenResolution a categoría
train_tb["ScreenResolution"] = train_tb["ScreenResolution"].str.split(
    " ").str[-1].astype("category")

# Crea una nueva columna llamada GHz con los GHz de la Cpu
train_tb["GHz"] = train_tb["Cpu"].str.split(
    " ").str[-1].replace(r'(GHz)$', "", regex=True).astype("float")

# Cambia el tipo de datos de la columna Cpu y elimina los GHz
train_tb["Cpu"] = train_tb["Cpu"].str.replace(
    r'(\d+(\.\d{1,2})?)GHz', "", regex=True).astype("category")

# Cambia el tipo de datos de la columna Gpu
train_tb["Gpu"] = train_tb["Gpu"].astype("category")

# Cambia el tipo de datos de la columna Ram y elimina la unidad
train_tb["Ram(GB)"] = train_tb["Ram"].str.replace(
    r'(GB)$', "", regex=True).astype("int")
train_tb.drop("Ram", axis=1, inplace=True)

# Cambia el tipo de datos de la columna OpSys
train_tb["OpSys"] = train_tb["OpSys"].astype("category")

# Crea una nueva columna llamada TypeMemory que determina el tipo de memoria
TypeMemory = train_tb['Memory'].str.extract(r'\b(HDD|SSD|Flash Storage|Hybrid)\b(?:.*?\b(HDD|SSD|Flash storage|Hybrid)\b)?')
TypeMemory.columns =["TypeMemory1", "TypeMemory2"]
TypeMemory["TypeMemory2"].fillna("unknown", inplace=True)
TypeMemory = TypeMemory.astype("category")
train_tb = pd.concat([train_tb, TypeMemory], axis=1)


# Cambia el tipo de datos de la columna Memory y elimina la unidad. Además aparece unicamente el número
train_tb["Memory1(GB)"] = train_tb['Memory'].str.findall(r'\d+(?:\.\d+)?(?:GB|TB)?(?:\+\d+(?:\.\d+)?(?:GB|TB)?)?').str[0].str.replace(
    r'(GB)$', "", regex=True).str.replace(r'(TB)$', "000", regex=True).str.replace('.', "").astype("int")
train_tb["Memory2(GB)"] = train_tb['Memory'].str.findall(r'\d+(?:\.\d+)?(?:GB|TB)?(?:\+\d+(?:\.\d+)?(?:GB|TB)?)?').str[1].str.replace(
    r'(GB)$', "", regex=True).str.replace(r'(TB)$', "000", regex=True).str.replace('.', "").fillna(0).astype("int")
train_tb.drop("Memory", axis=1, inplace=True)

# Cambia el tipo de datos de la columna Weight y elimina la unidad
train_tb["Weight"] = train_tb["Weight"].str.replace(
    r'(kg)$', "", regex=True).astype("float")

# Elimina las columnas que tengan valores nulos
train_tb.dropna(inplace=True, axis=1, how="all")

# Elimina la columna la columna Product porque es str
train_tb.drop("Product", axis=1, inplace=True)


In [14]:
# Importamos el DataFrame
X_pred = pd.read_csv("data/test.csv")

# Establece la columna laptop_ID como índice
X_pred.set_index("laptop_ID", inplace=True)

# Cambia el tipo de datos de la columna Company
X_pred["Company"] = X_pred["Company"].astype("category")

# Cambia el tipo de datos de la columna TypeName
X_pred["TypeName"] = X_pred["TypeName"].astype("category")

# Crea una nueva columna llamada Touchscreen que determina aquellos laptops que tienen Touchscreen
X_pred["Touchscreen"] = X_pred["ScreenResolution"].str.contains(
    ".*Touchscreen.*")
X_pred["Touchscreen"] = X_pred["Touchscreen"].astype("int")

# Crea dos columnas con la resolución por ancho y por altura de la pantalla llamadas ScreenResolutionWidth y ScreenResolutionHeight respectivamente. Elimina la columna ScreenResolution
X_pred["ScreenResolution"] = X_pred["ScreenResolution"].str.split(
    " ").str[-1].astype("category")

# Crea una nueva columna llamada GHz con los GHz de la Cpu
X_pred["GHz"] = X_pred["Cpu"].str.split(
    " ").str[-1].replace(r'(GHz)$', "", regex=True).astype("float")

# Cambia el tipo de datos de la columna Cpu y elimina los GHz
X_pred["Cpu"] = X_pred["Cpu"].str.replace(
    r'(\d+(\.\d{1,2})?)GHz', "", regex=True).astype("category")

# Cambia el tipo de datos de la columna Gpu
X_pred["Gpu"] = X_pred["Gpu"].astype("category")

# Cambia el tipo de datos de la columna Ram y elimina la unidad
X_pred["Ram(GB)"] = X_pred["Ram"].str.replace(
    r'(GB)$', "", regex=True).astype("int")
X_pred.drop("Ram", axis=1, inplace=True)

# Cambia el tipo de datos de la columna OpSys
X_pred["OpSys"] = X_pred["OpSys"].astype("category")

# Crea una nueva columna llamada TypeMemory que determina el tipo de memoria
TypeMemory = X_pred['Memory'].str.extract(r'\b(HDD|SSD|Flash Storage|Hybrid)\b(?:.*?\b(HDD|SSD|Flash storage|Hybrid)\b)?')
TypeMemory.columns =["TypeMemory1", "TypeMemory2"]
TypeMemory["TypeMemory2"].fillna("unknown", inplace=True)
TypeMemory = TypeMemory.astype("category")
X_pred = pd.concat([X_pred, TypeMemory], axis=1)


# Cambia el tipo de datos de la columna Memory y elimina la unidad. Además aparece unicamente el número
X_pred["Memory1(GB)"] = X_pred['Memory'].str.findall(r'\d+(?:\.\d+)?(?:GB|TB)?(?:\+\d+(?:\.\d+)?(?:GB|TB)?)?').str[0].str.replace(
    r'(GB)$', "", regex=True).str.replace(r'(TB)$', "000", regex=True).str.replace('.', "").astype("int")
X_pred["Memory2(GB)"] = X_pred['Memory'].str.findall(r'\d+(?:\.\d+)?(?:GB|TB)?(?:\+\d+(?:\.\d+)?(?:GB|TB)?)?').str[1].str.replace(
    r'(GB)$', "", regex=True).str.replace(r'(TB)$', "000", regex=True).str.replace('.', "").fillna(0).astype("int")
X_pred.drop("Memory", axis=1, inplace=True)

# Cambia el tipo de datos de la columna Weight y elimina la unidad
X_pred["Weight"] = X_pred["Weight"].str.replace(
    r'(kg)$', "", regex=True).astype("float")

# Elimina las columnas que tengan valores nulos
X_pred.dropna(inplace=True, axis=1, how="all")

# Elimina la columna la columna Product porque es str
X_pred.drop("Product", axis=1, inplace=True)

In [15]:
# Pasamos todas las columnas categóricas a columnas numéricas por categoría
train_tb, X_pred = fn.apply_label_encoder(train_tb, X_pred)

-----------------------------------------------------------------------------------------------------------------

## Pasos que si o si debéis realizar para poder participar en la competición:

### 1. Definir X e y

In [16]:
X = train_tb.drop(['Price_euros'], axis=1)
y = train_tb['Price_euros'].copy()
X.shape

(912, 15)

In [17]:
y.shape

(912,)

### 2. Dividir X_train, X_test, y_train, y_test

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [19]:
X_train

,Company,TypeName,Inches,ScreenResolution,Cpu,Gpu,OpSys,Weight,Touchscreen,GHz,Ram(GB),TypeMemory1,TypeMemory2,Memory1(GB),Memory2(GB)
laptop_ID,,,,,,,,,,,,,,,
26,4,3,15.6,0,47,47,5,2.30,0,2.0,4,1,3,1000,0
776,4,1,15.6,14,69,89,5,2.72,1,2.6,16,3,0,128,1000
1263,2,1,15.6,3,77,76,5,2.50,0,2.8,16,3,0,256,1000
182,10,3,15.6,3,57,4,5,1.90,0,2.5,8,3,0,128,1000
321,7,0,13.3,14,81,61,5,1.29,1,1.8,16,3,3,1000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,4,5,15.6,3,71,101,5,2.00,0,2.7,8,3,3,256,0
441,10,3,15.6,3,79,104,5,2.50,0,2.9,16,3,3,512,0
955,2,3,17.3,2,87,66,5,2.80,0,1.6,4,1,3,1000,0


### 3. Asignar el modelo (vacío) a una variable


In [20]:
model = LinearRegression()

In [21]:
model.fit(X_train, y_train)

LinearRegression()

In [22]:
prediction = model.predict(X_test)
prediction

array([1308.97760347,   38.65905995,  809.70185787, 1314.3965124 ,
       1815.4421665 ,  378.75101092,  834.68830589, 1800.9279722 ,
        940.27228671, 1252.97917056,  906.59328886,  513.50666525,
        162.4861034 , 1516.63103356, 1336.89461614, 1738.65777533,
       1776.84025904,  867.66288096, 1007.95753612,  781.28421099,
        824.93392847,  387.05571812, 1070.32359839, 1252.58694674,
       1250.51053429,  514.79555877,  983.97621471, 1918.63709901,
       1179.52332691,  641.17268092, 1408.06219791,  954.18992734,
        734.53422044, 1434.8380516 , 1254.96186539,  419.55398225,
        973.90882968, 1023.0635031 , 1261.84185746,  922.22201284,
       1031.27748839, 1559.63602502, 1056.29354992, 2210.22917108,
       1301.31292255,  291.98120764, 1208.16149772,  510.53890941,
       1150.24251416,  800.57238123, 1906.2655786 , 1214.98798396,
        297.86617258, 2032.59317437,  975.27805178, 1390.40791285,
       1895.36630514,  983.46442703, 2125.34592317, 1721.53661

### 4. Sacar métricas, valorar el modelo 

Recuerda que en la competición se va a evaluar con la métrica de MAE.

In [23]:
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, prediction))}")

RMSE: 394.7800430932667


-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir con el dataset de predicción 

Definición de **modelo que está listo**. 

Tras hacer suficientes pruebas, analizar los datos, hacer feature engineering, probar diferentes modelos con diferentes parámetros, es con este con el que observo mejores métricas y menos overfitting. ¡Cuidado con el overfitting aquí! Si vuestro modelo aprende muy bien de estos datos pero hay overfitting cuando le pasemos los datos desconocidos de `test.csv` nos arriesgamos a que digamos, no salga lo esperado.

### 1. Entrena dicho modelo con TODOS tus datos de train, esto es con `train.csv` al completo.


**CON LAS TRANSFORMACIONES QUE LE HAYAS REALIZADO A `X` INCLUÍDAS.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

In [24]:
modelo_total = LinearRegression()
modelo_total.fit(X, y)


LinearRegression()

### 2. Carga los datos de `test.csv` para predecir.

In [25]:
# X_pred = pd.read_csv("data/test.csv")
X_pred.head()

,Company,TypeName,Inches,ScreenResolution,Cpu,Gpu,OpSys,Weight,Touchscreen,GHz,Ram(GB),TypeMemory1,TypeMemory2,Memory1(GB),Memory2(GB)
laptop_ID,,,,,,,,,,,,,,,
750,16,3,14.0,3,57,53,5,1.25,1,2.50,8,3,3,256,0
726,12,3,14.0,4,25,41,5,1.40,0,1.92,4,1,3,32,0
633,12,3,14.0,3,27,40,5,1.40,0,1.44,2,0,3,32,0
363,4,1,15.6,3,58,75,5,2.65,0,2.50,8,1,3,1000,0
319,2,0,11.6,0,33,43,5,1.10,1,1.10,2,0,3,32,0


In [26]:
X_pred.shape

(391, 15)

In [27]:
X_pred.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 391 entries, 750 to 924
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           391 non-null    int64  
 1   TypeName          391 non-null    int64  
 2   Inches            391 non-null    float64
 3   ScreenResolution  391 non-null    int64  
 4   Cpu               391 non-null    int64  
 5   Gpu               391 non-null    int64  
 6   OpSys             391 non-null    int64  
 7   Weight            391 non-null    float64
 8   Touchscreen       391 non-null    int64  
 9   GHz               391 non-null    float64
 10  Ram(GB)           391 non-null    int64  
 11  TypeMemory1       391 non-null    int64  
 12  TypeMemory2       391 non-null    int64  
 13  Memory1(GB)       391 non-null    int64  
 14  Memory2(GB)       391 non-null    int64  
dtypes: float64(3), int64(12)
memory usage: 48.9 KB


**¿Por qué da error?** 

In [28]:
predictions_submit = model.predict(X_pred)
predictions_submit

array([ 1.36898190e+03,  6.41187724e+02,  3.46327685e+02,  1.00846461e+03,
        1.85799586e+02,  5.98740218e+02,  4.53634616e+02,  1.77497104e+03,
        1.83453913e+03,  1.42645965e+03,  1.03425914e+03,  1.24311516e+03,
        1.15967494e+03,  1.02167864e+03,  6.77643872e+02,  1.04171190e+03,
        1.12852734e+03,  1.95171319e+03,  7.04701733e+02,  1.17347529e+03,
        1.27774493e+03,  1.24957764e+03,  4.64574737e+02,  9.61012906e+02,
        2.99211163e+02,  1.54275353e+03,  7.19162572e+02,  1.02438526e+03,
        5.38827470e+02,  1.41405122e+03,  1.19284947e+03,  1.27699047e+03,
        1.09212552e+03,  1.27176390e+03,  1.48508246e+03,  1.30305610e+03,
        1.36111056e+03,  1.50611748e+03,  9.24807271e+02,  2.02079734e+03,
        1.79582497e+03,  1.25292415e+03,  2.07264126e+02,  1.82906191e+03,
        1.02762740e+03,  1.98443466e+03,  7.48054926e+02,  1.79449573e+03,
        1.07544054e+03,  1.62710420e+03,  8.39019203e+02,  3.35996703e+02,
        9.15892683e+02,  

IMPORTANTE: APLICAR LO MISMO A ESTOS DATOS QUE HAYÁIS APLICADO A LOS DATOS DE ENTRENAMIENTO

- SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 4 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS
- SI AL ARRAY CON EL QUE HICISTEIS `.fit()` LO NORMALIZASTEIS, PARA `.predict()` DEBÉIS NORMALIZARLO
- TODO IGUAL SALVO BORRAR FILAS, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER 391 FILAS, SI O SI

**Entonces, si al cargar los datos de train usé `index_col=0` para que utilizara la primera columna del conjunto de datos como índice, ¿tendré que hacerlo también para el conjunto `test.csv`?**

In [29]:
# ¿Qué opináis?
# Sí

![](https://www.tierraljelechu.com/web/img/wiki_up/1.996-SorpresaDto.-1-Red.jpg)

In [30]:
# X_pred = pd.read_csv("data/test.csv", index_col=0)
# X_pred.head()

### 3. AHORA puedo hacer la predicción que será lo que subirás a Kaggle. 

**¿Qué es lo que subirás a Kaggle?**

In [31]:
predictions_submit = model.predict(X_pred)
predictions_submit

array([ 1.36898190e+03,  6.41187724e+02,  3.46327685e+02,  1.00846461e+03,
        1.85799586e+02,  5.98740218e+02,  4.53634616e+02,  1.77497104e+03,
        1.83453913e+03,  1.42645965e+03,  1.03425914e+03,  1.24311516e+03,
        1.15967494e+03,  1.02167864e+03,  6.77643872e+02,  1.04171190e+03,
        1.12852734e+03,  1.95171319e+03,  7.04701733e+02,  1.17347529e+03,
        1.27774493e+03,  1.24957764e+03,  4.64574737e+02,  9.61012906e+02,
        2.99211163e+02,  1.54275353e+03,  7.19162572e+02,  1.02438526e+03,
        5.38827470e+02,  1.41405122e+03,  1.19284947e+03,  1.27699047e+03,
        1.09212552e+03,  1.27176390e+03,  1.48508246e+03,  1.30305610e+03,
        1.36111056e+03,  1.50611748e+03,  9.24807271e+02,  2.02079734e+03,
        1.79582497e+03,  1.25292415e+03,  2.07264126e+02,  1.82906191e+03,
        1.02762740e+03,  1.98443466e+03,  7.48054926e+02,  1.79449573e+03,
        1.07544054e+03,  1.62710420e+03,  8.39019203e+02,  3.35996703e+02,
        9.15892683e+02,  

**¡PERO! Para subir a Kaggle la predicción, ésta tendrá que tener una forma específica y no valdrá otra.**

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [32]:
sample = pd.read_csv("data/sample_submission.csv")

In [33]:
sample.head()

,laptop_ID,Price_euros
0,750,500
1,726,500
2,633,500
3,363,650
4,319,650


In [34]:
sample.shape

(391, 2)

### 4. Mete tus predicciones en un dataframe. 

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [35]:
submission = pd.DataFrame({"laptop_ID": sample["laptop_ID"], "Price_euros": predictions_submit})

In [36]:
submission.head()

,laptop_ID,Price_euros
0,750,1368.981898
1,726,641.187724
2,633,346.327685
3,363,1008.464613
4,319,185.799586


In [37]:
submission.shape

(391, 2)

### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [38]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.laptop_ID.all() == sample.laptop_ID.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")


In [39]:
chequeator(submission)

You're ready to submit!


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1076)>

In [40]:
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, prediction))}")

RMSE: 394.7800430932667
